In [1]:
import pandas as pd
import json

In [2]:
# === BƯỚC 1: ĐỌC DỮ LIỆU TỪ TỆP CSV ===
# (Phần này thay thế cho list `disease_data` cũ)

# Tên tệp CSV bạn đã tải lên
input_csv_file = "../data/raw/raw_data_bayes.csv"

# List này sẽ chứa dữ liệu được tải từ CSV
disease_data = []

try:
    # Đọc tệp CSV
    df_raw = pd.read_csv(input_csv_file)
    
    # Kiểm tra tên cột (dựa trên tệp của bạn là 'disease' và 'sympton')
    if 'disease' not in df_raw.columns or 'sympton' not in df_raw.columns:
        print(f"Lỗi: Tệp CSV phải có cột 'disease' và 'sympton'.")
        print(f"Tên cột tìm thấy: {df_raw.columns.tolist()}")
    
    else:
        # Duyệt qua từng dòng trong tệp CSV
        for index, row in df_raw.iterrows():
            disease = row['disease']
            symptoms_str = str(row['sympton']) # Đảm bảo là string
            
            # Bỏ qua nếu dòng bị thiếu dữ liệu
            if pd.isna(disease) or pd.isna(symptoms_str) or not symptoms_str:
                continue
            
            # Tách chuỗi triệu chứng thành một danh sách (list)
            # Ví dụ: "sốt, đau_đầu, mệt_mỏi"
            symptoms_list_raw = symptoms_str.split(',')
            
            # Làm sạch: loại bỏ khoảng trắng thừa
            # ["sốt", " đau_đầu", " mệt_mỏi"] -> ["sốt", "đau_đầu", "mệt_mỏi"]
            symptoms_list_clean = [symptom.strip() for symptom in symptoms_list_raw if symptom.strip()]
            
            # Thêm vào `disease_data` theo định dạng (list, string)
            if symptoms_list_clean: # Chỉ thêm nếu có triệu chứng
                disease_data.append( (symptoms_list_clean, disease) )
        
        print(f"✓ Đã đọc và xử lý thành công {len(disease_data)} dòng từ '{input_csv_file}'")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp '{input_csv_file}'")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc tệp: {e}")

# === HẾT PHẦN ĐỌC DỮ LIỆU ===

✓ Đã đọc và xử lý thành công 139 dòng từ '../data/raw/raw_data_bayes.csv'


In [3]:
# 3_prepare_naive_bayes_data.py
# Chuẩn bị dữ liệu cho Naive Bayes Classifier


# === CÁC PHẦN CÒN LẠI CỦA SCRIPT (Giữ nguyên) ===
# Script sẽ tiếp tục chạy với `disease_data` đã được tải từ CSV

if disease_data: # Chỉ chạy nếu dữ liệu được tải thành công
    
    # Tạo danh sách tất cả triệu chứng có thể có
    all_symptoms = set()
    for symptoms, _ in disease_data:
        all_symptoms.update(symptoms)

    all_symptoms = sorted(list(all_symptoms))

    print(f"\n📊 Thống kê dữ liệu:")
    print(f"   - Số mẫu: {len(disease_data)}")
    print(f"   - Số triệu chứng: {len(all_symptoms)}")
    print(f"   - Số bệnh: {len(set([d for _, d in disease_data]))}")

    # Chuyển sang binary format
    binary_data = []
    for symptoms, disease in disease_data:
        row = [1 if symptom in symptoms else 0 for symptom in all_symptoms]
        row.append(disease)
        binary_data.append(row)

    # Tạo DataFrame
    columns = all_symptoms + ["disease"]
    df_binary = pd.DataFrame(binary_data, columns=columns)

    # Lưu file
    output_bayes_csv = "../data/processed/naive_bayes_data.csv"
    df_binary.to_csv(output_bayes_csv, index=False, encoding="utf-8-sig")
    print(f"\n✓ Đã lưu file dữ liệu nhị phân '{output_bayes_csv}'")

    # Lưu mapping triệu chứng
    output_mapping_json = "../data/processed/symptom_mapping.json"
    symptom_mapping = {
        "all_symptoms": all_symptoms,
        "diseases": sorted(list(set([d for _, d in disease_data])))
    }

    with open(output_mapping_json, "w", encoding="utf-8") as f:
        json.dump(symptom_mapping, f, ensure_ascii=False, indent=2)

    print(f"✓ Đã lưu file mapping '{output_mapping_json}'")

    # Hiển thị mẫu
    print("\n📋 Dữ liệu mẫu (5 dòng đầu của tệp nhị phân):")
    print(df_binary.head())

    print(f"\n📋 Danh sách triệu chứng ({len(all_symptoms)}):")
    # In một vài triệu chứng
    for i, symptom in enumerate(all_symptoms[:10], 1): # In 10 cái đầu
        print(f"   {i}. {symptom}")
    if len(all_symptoms) > 10:
        print(f"   ... và {len(all_symptoms) - 10} triệu chứng khác.")


    print("\n📋 Danh sách bệnh:")
    diseases = df_binary['disease'].unique()
    for i, disease in enumerate(diseases, 1):
        count = len(df_binary[df_binary['disease'] == disease])
        print(f"   {i}. {disease} ({count} mẫu)")
else:
    print("\nKhông có dữ liệu để xử lý. Vui lòng kiểm tra lại tệp CSV đầu vào.")


📊 Thống kê dữ liệu:
   - Số mẫu: 139
   - Số triệu chứng: 219
   - Số bệnh: 37

✓ Đã lưu file dữ liệu nhị phân '../data/processed/naive_bayes_data.csv'
✓ Đã lưu file mapping '../data/processed/symptom_mapping.json'

📋 Dữ liệu mẫu (5 dòng đầu của tệp nhị phân):
   SYMPTON  amidan_sưng  bong_vảy  buồn_bã_kéo_dài  buồn_nôn  bồn_chồn  \
0        0            0         0                0         0         0   
1        0            0         0                0         0         0   
2        0            0         0                0         0         0   
3        0            0         0                0         0         0   
4        0            0         0                0         0         0   

   chán_ăn  chóng_mặt  chướng_bụng  chảy_ghèn_vàng  ...  đổ_mồ_hôi  \
0        0          0            0               0  ...          0   
1        0          0            0               0  ...          0   
2        0          0            0               0  ...          0   
3        0   